In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
from dotenv import load_dotenv
import nest_asyncio
import os

os.environ["SWE_AGENT_CONFIG_DIR"] = "."
os.environ["SWE_AGENT_TOOLS_DIR"] = "tools"
os.environ["SWE_AGENT_TRAJECTORY_DIR"] = "trajectories"

os.makedirs("replays", exist_ok=True)
os.makedirs("trajectories", exist_ok=True)

load_dotenv()
nest_asyncio.apply()

In [4]:
from instances import (
    as_instances_iter,
    get_filtered_swe_smith_instances_df,
    get_swe_bench_verified_instances_df,
)

# instances_df = get_filtered_swe_smith_instances_df()
instances_df = get_swe_bench_verified_instances_df()
instances_iter = as_instances_iter(instances_df)
for instance in instances_iter:
    # break
    if (
        # instance["difficulty"] == "<15 min fix"
        instance["repo"] not in ("astropy/astropy", "django/django", "matplotlib/matplotlib")
        # and instance["instance_id"] != "django__django-10097"
    ):
        break
instance

{'repo': 'mwaskom/seaborn',
 'instance_id': 'mwaskom__seaborn-3069',
 'base_commit': '54cab15bdacfaa05a88fbc5502a5b322d99f148e',
 'patch': 'diff --git a/seaborn/_core/plot.py b/seaborn/_core/plot.py\n--- a/seaborn/_core/plot.py\n+++ b/seaborn/_core/plot.py\n@@ -25,7 +25,7 @@\n from seaborn._stats.base import Stat\n from seaborn._core.data import PlotData\n from seaborn._core.moves import Move\n-from seaborn._core.scales import Scale\n+from seaborn._core.scales import Scale, Nominal\n from seaborn._core.subplots import Subplots\n from seaborn._core.groupby import GroupBy\n from seaborn._core.properties import PROPERTIES, Property\n@@ -1238,7 +1238,6 @@ def _setup_scales(\n             # This only affects us when sharing *paired* axes. This is a novel/niche\n             # behavior, so we will raise rather than hack together a workaround.\n             if axis is not None and Version(mpl.__version__) < Version("3.4.0"):\n-                from seaborn._core.scales import Nominal\n        

In [5]:
import art
from rollout import ModelConfig, rollout

model = art.Model(
    # name="openrouter/google/gemini-2.5-flash-preview-05-20",
    name="openrouter/anthropic/claude-3.5-haiku",
    project="sweagent",
    config=ModelConfig(max_input_tokens=131072),
)

trajectory, run_single = await rollout(
    model,
    instance,
    # replay_trajectory_path=Path(
    #     "/Users/brad/github/OpenPipe/agent-reinforcement-training/dev/sweagent/trajectories/brad/no_config__openrouter/google/gemini-2.5-flash-preview-05-20__t-1.00__p-1.00__c-0.00___c01334/c01334/c01334.traj"
    # ),
)

Please make sure your output includes exactly _ONE_ function call!
You must invoke the function directly using the function call format.
You cannot invoke commands with ```, you have to use the function call format.
If you think you have already resolved the issue, please submit your changes by running the `submit` command.
If you think you cannot solve the problem, please run `exit_forfeit` (if available) or `submit`.
Else, please continue with a new tool call!


╭──────────────────────────── 🎉 Submission successful 🎉 ────────────────────────────╮
│ SWE-agent has produced a patch that it believes will solve the issue you submitted! │
│ Use the code snippet below to inspect or apply it!                                  │
╰─────────────────────────────────────────────────────────────────────────────────────╯

                                                                                                                   
 # The patch has been saved to your local filesystem at:                                                           
 PATCH_FILE_PATH='/Users/brad/github/OpenPipe/agent-reinforcement-training/dev/swebench/trajectories/brad/no_confi 
 _openrouter/anthropic/claude-3.5-haiku__t-1.00__p-1.00__c-0.00___739f9b/739f9b/739f9b.patch'                      
 # Inspect it:                                                                                                     
 cat "${PATCH_FILE_PATH}"                                                                                          
 # Apply it to a local repository:                                                                                 
 cd <your local repo root>                                                                                         
 git apply "${PATCH_FILE_PATH}"                                                                                    
                                                                                                                   